In [1]:
import io
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import keras.backend as K
import keras as k

c:\Users\dream\anaconda3\envs\default\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\dream\anaconda3\envs\default\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
I

In [14]:
def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)

train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

# Build your input pipelines
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.map(_normalize_img)

test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.map(_normalize_img)

In [17]:
tfds.as_numpy(test_dataset)

In [3]:
(x_train_master, y_train_master), (x_test_master, y_test_master) = k.datasets.fashion_mnist.load_data()
x_train_master = x_train_master /  255
x_test_master = x_test_master / 255

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 32)          0

In [5]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [13]:
print(x_train_master.shape, y_train_master.shape)

(60000, 28, 28) (60000,)


In [6]:
# Train the network
history = model.fit(
    x_train_master, y_train_master, batch_size=64,
    epochs=10,)

Epoch 1/10
938/938 [==============================] - 10s 8ms/step - loss: 0.7269
Epoch 2/10
938/938 [==============================] - 7s 7ms/step - loss: 0.6214
Epoch 3/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5901
Epoch 4/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5693
Epoch 5/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5574
Epoch 6/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5500
Epoch 7/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5433
Epoch 8/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5362
Epoch 9/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5316
Epoch 10/10
938/938 [==============================] - 7s 7ms/step - loss: 0.5315


In [7]:
# Evaluate the network
results = model.predict(x_test_master)

313/313 [==============================] - 0s 1ms/step


In [18]:
results

array([[ 0.04670693,  0.02187376,  0.10099864, ..., -0.05968593,
         0.02966552, -0.0842853 ],
       [-0.04415048,  0.0667732 , -0.03544863, ..., -0.01037479,
         0.04544495, -0.00181162],
       [ 0.08167237,  0.0037541 , -0.00276175, ..., -0.05668079,
         0.05225655,  0.00915951],
       ...,
       [-0.04512468,  0.02758452,  0.0766301 , ...,  0.06048002,
         0.04214891,  0.10131395],
       [ 0.07498088, -0.00552523, -0.00225884, ..., -0.05290893,
         0.04841228,  0.01570987],
       [ 0.00757757, -0.06552477, -0.06683901, ...,  0.11751451,
         0.01972228, -0.00483689]], dtype=float32)

In [11]:
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
# for img, labels in tfds.as_numpy(test_dataset):
#     [out_m.write(str(x) + "\n") for x in labels]
# out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass